In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string

In [2]:
df = pd.read_csv("emoji_df.csv")
df.head()

,emoji,name,group,sub_group,codepoints
0,😀,grinning face,Smileys & Emotion,face-smiling,1F600
1,😃,grinning face with big eyes,Smileys & Emotion,face-smiling,1F603
2,😄,grinning face with smiling eyes,Smileys & Emotion,face-smiling,1F604
3,😁,beaming face with smiling eyes,Smileys & Emotion,face-smiling,1F601
4,😆,grinning squinting face,Smileys & Emotion,face-smiling,1F606


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/megha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/megha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [96]:
# preprocess text
def preprocess(text):
    tokens = word_tokenize(text)
    
    # remove punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens

In [7]:
groups = df['group'].unique()
sub_groups = df['sub_group'].unique()

In [10]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

In [24]:
word = "apple"
hypothesis_template = 'This text is about {}.'
prediction = classifier(word, sub_groups, hypothesis_template=hypothesis_template, multi_label=True)
prediction

{'sequence': 'apple',
 'labels': ['food-fruit',
  'food-sweet',
  'drink',
  'food-vegetable',
  'food-asian',
  'event',
  'other-object',
  'person',
  'plant-other',
  'animal-amphibian',
  'emotion',
  'place-other',
  'warning',
  'musical-instrument',
  'av-symbol',
  'person-activity',
  'animal-mammal',
  'music',
  'tool',
  'food-prepared',
  'computer',
  'person-role',
  'family',
  'sound',
  'place-geographic',
  'face-neutral-skeptical',
  'time',
  'household',
  'person-gesture',
  'person-symbol',
  'light & video',
  'phone',
  'body-parts',
  'game',
  'money',
  'place-building',
  'mail',
  'face-affection',
  'office',
  'face-concerned',
  'animal-bird',
  'writing',
  'sport',
  'person-sport',
  'science',
  'animal-reptile',
  'face-negative',
  'heart',
  'hotel',
  'person-resting',
  'gender',
  'punctuation',
  'face-unwell',
  'currency',
  'face-smiling',
  'keycap',
  'transport-ground',
  'award-medal',
  'transport-sign',
  'sky & weather',
  'medica

In [31]:
# get most likely group based on a word
def get_group(word, groups):
    hypothesis_template = 'This text is about {}.'
    prediction = classifier(word, groups, hypothesis_template=hypothesis_template, multi_label=True)
    return [prediction['labels'][0], prediction['scores'][0]]

get_group('apple',groups)

['Food & Drink', 0.3914111852645874]

In [111]:
def word_to_emoji(word):
    # first look for direct matches
    if word in df.loc[df['name'] == word, 'name'].values:
        return df.loc[df['name'] == word, 'emoji'].values[0]

    group = get_group(word, groups)
    # print("group:",group)

    # option 1
    # possible_subgroups = df[df['group'] == group[0]]['sub_group'].unique()
    # print("possible subgroups:",possible_subgroups)
    # subgroup = get_group(word, possible_subgroups)
    # print("subgroup:",subgroup)
    # possible_emojis = df[df['sub_group'] == subgroup[0]]['name'].unique()

    # option 2
    possible_emojis = df[df['group'] == group[0]]['name'].unique()

    # print("possible emojis:",possible_emojis)
    name = get_group(word,possible_emojis)
    # print(name)
    return [df[df['name'] == name[0]].iloc[0]['emoji'], name[1]]

# print(word_to_emoji('happy'))

In [118]:
# convert input text to emojis
input = "the strawberry is on fire!"
print('input text:',input)

input = preprocess(input)
# print('preprocessed input text:',input)

emoji_translation = ''
for word in input:
    emoji = word_to_emoji(word)
    emoji_translation = emoji_translation + emoji[0]
    # print(emoji)

print('emoji translation:',emoji_translation)

input text: the strawberry is on fire!
emoji translation: 🍓🔥
